<a href="https://colab.research.google.com/github/Br-alexis73/Face_recognition/blob/master/fr_casia_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'casia-face-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4502072%2F7710309%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240420%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240420T222148Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D63e9d4210e230cac835476e827d8feb252647573aa6956d25737a1a3da52b7a473e3fdbe7b82c96e68971db2ddb6ca22ae18c018ceaa74d4febc534e9c7316e1028a53f2d1b54dbe082940661ae0138f20dbb1ba05603b5ab7eb4c5b5c946e834a42d226bc80372359c5c39db97c76e0a8c2a953cd452d5cf2456505a3e0470c22f8c2c7e0a4b69c72ce722a26232ee46eaad8009f62ccf1fcffb3a38a045e8b3d94a91c23770c2354150d5265447792467bac3814c51de2a5cd35bcc97fec90523ef71d0eca6c5ab496335290f5d1cd8d300cc3c998f180720ce18bab96bf8b98ade4f089df20b2cb707156c8395502a409eaf5ce2d71c1329a078f5c7837e0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle

# Set the dataset path to your Kaggle dataset location
dataset_path = '../input/casia-face-dataset/casia dataset'

2024-04-20 22:15:12.600380: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 22:15:12.600514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 22:15:12.779846: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Generate a list of (id, image count) tuples
ids_images = []

# Loop over each person's directory
for person_id in os.listdir(dataset_path):
    directory = os.path.join(dataset_path, person_id)
    if os.path.isdir(directory):
        images = [i for i in os.listdir(directory) if i.endswith('.jpg')]
        image_count = len(images)
        # Add to list if there are a minimum number of images (e.g., 50)
        if image_count >= 50:
            ids_images.append((person_id, min(image_count, 50)))

# Create a DataFrame with ids and image counts
people_df = pd.DataFrame(ids_images, columns=['id', 'images'])

In [ ]:
# Define image size for CNN input
im_size = 224

# Initialize lists to store the images and labels
images = []
labels = []

# Load and preprocess images
for person_id, _ in ids_images:
    person_directory = os.path.join(dataset_path, person_id)
    person_images = sorted([os.path.join(person_directory, i) for i in os.listdir(person_directory) if i.endswith('.jpg')])[:50]
    for image_path in person_images:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (im_size, im_size))
        img = img.astype('float32') / 255.0
        images.append(img)
        labels.append(person_id)
#
# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Shuffle and split dataset
images, labels = shuffle(images, labels, random_state=1)
train_x, test_x, train_y, test_y = train_test_split(images, y_onehot, test_size=0.3, random_state=415)

# Output shapes to verify
print("train_x shape:", train_x.shape)
print("train_y shape:", train_y.shape)
print("test_x shape:", test_x.shape)
print("test_y shape:", test_y.shape)

NameError: name 'y_onehot' is not defined